In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import t
#import pandas as pd


In [3]:


# to see variance of both samples pre and post-substitution
cols = ['old_document', 'new_document', 'old_fk_score','new_fk_score','old_fk_grade','new_fk_grade', 'old_dc_score',
        'new_dc_score','old_cl_score','new_cl_score','old_ar_index','new_ar_index', 'old_smog_score', 'new_smog_score',
        'old_gf_index', 'new_gf_index']

#'/home/karl/PycharmProjects/DLAI/translate/prold_pipeline_results_graded.txt' - for prog+ai
with open('/home/karl/PycharmProjects/DLAI_GPT/gpt-j-6b-main/pipeline_results_graded.txt', encoding='utf-8-sig') as data:
    df = pd.read_csv(data,  sep='|',names=cols).reset_index()
    df = df.drop(['index'], axis=1)
    df = df.drop([0])
    df['old_fk_score'] = df['old_fk_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_fk_score'] = df['new_fk_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['old_fk_grade'] = df['old_fk_grade'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_fk_grade'] = df['new_fk_grade'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['old_dc_score'] = df['old_dc_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_dc_score'] = df['new_dc_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['old_cl_score'] = df['old_cl_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_cl_score'] = df['new_cl_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['old_ar_index'] = df['old_ar_index'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_ar_index'] = df['new_ar_index'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['old_smog_score'] = df['old_smog_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_smog_score'] = df['new_smog_score'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['old_gf_index'] = df['old_gf_index'].apply(lambda col:pd.to_numeric(col, errors='coerce'))
    df['new_gf_index'] = df['new_gf_index'].apply(lambda col:pd.to_numeric(col, errors='coerce'))

    #x_simple = np.array(((df["new_fk_score"] - df["old_fk_score"]) / df['old_fk_score']) * 100)
    #y_simple = np.array(df["old_fk_score"])
    #n, bins, patches = plt.hist(y_simple, bins=100)
    #plt.show()
    #plt.savefig('/home/karl/Pictures/old.png')

    #x_simple = np.array(df["new_fk_score"])
    #n, bins, patches = plt.hist(x_simple, bins=100)
    #plt.show()
    #plt.savefig('/home/karl/Pictures/new.png')

    #my_rho = np.corrcoef(x_simple, y_simple)
    #print("Correlation = " + str(my_rho[0,1]))
    #fig, ax = plt.subplots()
    #ax.scatter(x_simple, y_simple)
    #ax.set_title("Correlation = " + str(my_rho[0,1]))
    #plt.show()
    #plt.savefig('/home/karl/Pictures/correlation.png')

# Here we see variance is about the same so the pipeline is effectively swapping the words needed and not
# swapping words, inappropriately per the rules of the pipeline. Additionally, we can use a t-test as variance is low.
# However, there is a small difference in SD for each sample pre-and-post sub, so we use a Welch T-test.

In [5]:
df2 = df
df2 = df2.drop(['old_document', 'new_document', 'old_dc_score', 'new_dc_score', 'old_cl_score', 'new_cl_score', 'old_smog_score', 'new_smog_score'], axis=1)
df2.to_csv('gpt_scores_table_for_box_plot.csv')

In [7]:
df

old_document        object
new_document        object
old_fk_score       float64
new_fk_score       float64
old_fk_grade       float64
new_fk_grade       float64
old_dc_score       float64
new_dc_score       float64
old_cl_score       float64
new_cl_score       float64
old_ar_index       float64
new_ar_index       float64
old_smold_score    float64
new_smold_score    float64
old_gf_index       float64
new_gf_index       float64
dtype: object

In [ ]:
df.dtypes

In [5]:
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t
import pandas as pd

def welch_ttest(x1, x2,alternative):
    std1 = x1.std()
    std2 = x2.std()
    n1 = x1.size
    n2 = x2.size
    m1 = np.mean(x1)
    m2 = np.mean(x2)

    v1 = np.var(x1, ddof=1)
    v2 = np.var(x2, ddof=1)

    pooled_se = np.sqrt(v1 / n1 + v2 / n2)
    delta = m1-m2

    tstat = delta /  pooled_se
    df = (v1 / n1 + v2 / n2)**2 / (v1**2 / (n1**2 * (n1 - 1)) + v2**2 / (n2**2 * (n2 - 1)))

    # two side t-test
    p = 2 * t.cdf(-abs(tstat), df)

    # upper and lower bounds
    lb = delta - t.ppf(0.975,df)*pooled_se
    ub = delta + t.ppf(0.975,df)*pooled_se

    result_list = [m2, m1, tstat,p,delta,lb,ub, std1, std2]
    return  result_list

In [6]:
#columns=['Pre-sub mean', 'Post-sub mean', 'T-statistic','p-value 2 sided','Difference in mean','CI lower bound','CI upper bound', 'Pre-sub STD', 'Post-sub STD'])


ts1 = df['new_fk_score']
ts2 = df["old_fk_score"]
fk_score_stats = welch_ttest(ts1,ts2,"equal")
fk_score_stats


[35.59366044985741,
 67.2306354978077,
 14.00548419460426,
 1.3749893157749472e-41,
 31.636975047950287,
 27.205448823043643,
 36.068501272856935,
 49.33637589411156,
 34.91992954550203]

In [7]:
ts3 = df['new_fk_grade']
ts4 = df["old_fk_grade"]
fk_grade_stats = welch_ttest(ts3,ts4,"equal")
fk_grade_stats

[10.752294288683798,
 5.8972302962476935,
 -13.986481264076321,
 1.990015003723007e-41,
 -4.855063992436104,
 -5.536071758633763,
 -4.174056226238446,
 7.6810908992565565,
 5.222647635382556]

In [8]:
ts1 = df['new_dc_score']
ts2 = df["old_dc_score"]
dc_score_stats = welch_ttest(ts1,ts2,"equal")
dc_score_stats

[12.527905237874613,
 14.627330327245245,
 14.033291292147268,
 6.043602669836232e-42,
 2.099425089370632,
 1.8059533907688317,
 2.392896787972432,
 3.05015194543767,
 2.5925759844972216]

In [9]:
ts1 = df['new_cl_score']
ts2 = df["old_cl_score"]
cl_score_stats = welch_ttest(ts1,ts2,"equal")
cl_score_stats

[-8.115293654786043,
 -41.63259287074375,
 -12.415900971448082,
 1.910734659422187e-33,
 -33.517299215957706,
 -38.81348790917936,
 -28.221110522736055,
 60.28609923529347,
 39.7928649803151]

In [10]:
ts1 = df['new_ar_index']
ts2 = df["old_ar_index"]
ar_score_stats= welch_ttest(ts1,ts2,"equal")
ar_score_stats

[12.703499952377719,
 2.3702420603910417,
 -26.358970525306525,
 4.077199695168121e-124,
 -10.333257891986676,
 -11.102285857267223,
 -9.56422992670613,
 8.18589531222071,
 6.559426266374294]

In [11]:
ts1 = df['new_smold_score']
ts2 = df["old_smold_score"]
smold_score_stats = welch_ttest(ts1,ts2,"equal")
smold_score_stats

[64.53037327684274,
 51.10196450195839,
 -10.570639949727353,
 3.9966969855257213e-25,
 -13.428408774884346,
 -15.920536690187095,
 -10.936280859581597,
 27.31387891113477,
 20.23423821963918]

In [12]:
ts1 = df['new_gf_index']
ts2 = df["old_gf_index"]
gf_score_stats = welch_ttest(ts1,ts2,"equal")
gf_score_stats



[23.107863430035497,
 19.299696583615304,
 -10.101895504264862,
 3.6490740612914545e-23,
 -3.8081668464201925,
 -4.547701053489555,
 -3.06863263935083,
 8.0772847699553,
 6.04223423021072]

In [13]:
d = {"GPT Approach": ['Pre-sub mean', 'Post-sub mean', 'T-statistic','p-value 2 sided','Difference in mean','CI lower bound','CI upper bound', 'Pre-sub STD', 'Post-sub STD'],
     "FK Score" : fk_score_stats,
     "FK Grade" : fk_grade_stats,
     "AR index" : ar_score_stats,
     "GF index" : gf_score_stats
}

df = pd.DataFrame(data=d).astype(str)
df

,GPT Approach,FK Score,FK Grade,AR index,GF index
0,Pre-sub mean,35.59366044985741,10.752294288683798,12.703499952377719,23.107863430035497
1,Post-sub mean,67.2306354978077,5.8972302962476935,2.3702420603910417,19.299696583615304
2,T-statistic,14.00548419460426,-13.986481264076321,-26.358970525306525,-10.101895504264862
3,p-value 2 sided,1.3749893157749472e-41,1.990015003723007e-41,4.077199695168121e-124,3.6490740612914545e-23
4,Difference in mean,31.636975047950287,-4.855063992436104,-10.333257891986676,-3.8081668464201925
5,CI lower bound,27.205448823043643,-5.536071758633763,-11.102285857267223,-4.547701053489555
6,CI upper bound,36.068501272856935,-4.174056226238446,-9.56422992670613,-3.06863263935083
7,Pre-sub STD,49.33637589411156,7.6810908992565565,8.18589531222071,8.0772847699553
8,Post-sub STD,34.91992954550203,5.222647635382556,6.559426266374294,6.04223423021072


In [14]:
df.to_csv("GPT_table2_for_paper.csv")